# Task Level(Advanced):

In [6]:
import pandas as pd

In [15]:
# Load the cricket fielding data
df = pd.read_csv('IPLsampledata.csv',skiprows=4)

In [18]:
#  Clean the column names 
# (This removes the 'Unnamed' labels and any accidental spaces)
df.columns = df.columns.str.strip()

# 3. Remove the first empty column if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# 4. Remove any rows that are completely empty
df = df.dropna(subset=['Player Name'])

# Now check your columns
print("Cleaned Columns:", df.columns.tolist())
print(df.head())

Cleaned Columns: ['Match No.', 'Innings', 'Teams', 'Player Name', 'BallCount', 'Position', 'Pick', 'Throw', 'Runs', 'Overcount', 'Venue', 'Stadium']
  Match No. Innings           Teams    Player Name BallCount  \
0   IPL2367       1  Delhi Capitals  Rilee russouw       0.1   
1   IPL2367       1  Delhi Capitals      Phil Salt       0.2   
2   IPL2367       1  Delhi Capitals     Yash Dhull       0.3   
3   IPL2367       1  Delhi Capitals     Axer Patel       0.4   
5   IPL2367       1  Delhi Capitals    Lalit yadav       0.6   

           Position Pick Throw Runs Overcount  Venue              Stadium  
0  Short mid wicket    n   NaN    1         1  Delhi  Arun Jaitly Stadium  
1     wicket keeper    Y     Y  NaN         1  Delhi  Arun Jaitly Stadium  
2            covers    Y     Y  NaN         1  Delhi  Arun Jaitly Stadium  
3             point    Y     Y  NaN         1  Delhi  Arun Jaitly Stadium  
5       cover point    Y     Y  NaN         1  Delhi  Arun Jaitly Stadium  


In [20]:
df_raw = pd.read_csv('IPL sample data.csv' if 'IPL sample data.csv' in locals() else 'IPLsampledata.csv', header=None)
print(df_raw.head(10))

      0                                                  1           2   \
0   Pick                                                Y->  Clean Pick   
1  Throw                                                Y->  Good Throw   
2   Runs  "+" stands for runs saved "-" stands for runs ...         NaN   
3    NaN                                                NaN         NaN   
4    NaN                                          Match No.     Innings   
5    NaN                                            IPL2367           1   
6    NaN                                            IPL2367           1   
7    NaN                                            IPL2367           1   
8    NaN                                            IPL2367           1   
9    NaN                                            IPL2367           1   

               3              4          5                 6     7   \
0             N->         Fumble        C->             Catch  DC->   
1             N->      Bad throw

In [21]:
# Drop the first unnamed column if it exists (it was NaN in the raw view)
if df.columns[0].startswith('Unnamed'):
    df = df.iloc[:, 1:]

# Cleaning player names (stripping whitespace)
df['Player Name'] = df['Player Name'].str.strip()

# Dropping rows where Player Name is NaN (some rows like row 9 in raw view were empty)
df = df.dropna(subset=['Player Name'])

# Filter for three specific players
# Looking at the data, let's pick Rilee russouw, Phil Salt, and Yash Dhull
players_to_analyze = ['Rilee russouw', 'Phil Salt', 'Yash Dhull']
df_filtered = df[df['Player Name'].isin(players_to_analyze)].copy()

# Map metrics
# Performance Score (PS) components
# Weights: CP=1, GT=1, C=3, DC=-3, ST=3, RO=3, MRO=-2, DH=2, RS=as is
# Pick legend: Y: CP, C: Catch, DC: Drop, S: Stumping, n/N: Fumble
# Throw legend: Y: GT, DH: Direct Hit, RO: Run Out, MR: Missed Runout, N: Bad throw

def calculate_score(row):
    score = 0
    pick = str(row['Pick']).strip().upper()
    throw = str(row['Throw']).strip().upper()
    runs = row['Runs']
    
    # Handle NaN runs
    if pd.isna(runs):
        runs = 0
    else:
        try:
            runs = float(runs)
        except:
            runs = 0
    
    # Pick Scores
    if pick == 'Y': score += 1 # Clean Pick
    elif pick == 'C': score += 3 # Catch
    elif pick == 'DC': score -= 3 # Dropped Catch
    elif pick == 'S': score += 3 # Stumping
    
    # Throw Scores
    if throw == 'Y': score += 1 # Good Throw
    elif throw == 'DH': score += 2 # Direct Hit
    elif throw == 'RO': score += 3 # Run Out
    elif throw == 'MR': score -= 2 # Missed Run Out
    
    # Runs Saved/Conceded
    score += runs
    
    return score

# Apply scoring
df_filtered['Ball_PS'] = df_filtered.apply(calculate_score, axis=1)

# Group by player to get final performance stats
summary = df_filtered.groupby('Player Name').agg(
    Clean_Picks=('Pick', lambda x: (x.str.strip().str.upper() == 'Y').sum()),
    Catches=('Pick', lambda x: (x.str.strip().str.upper() == 'C').sum()),
    Dropped_Catches=('Pick', lambda x: (x.str.strip().str.upper() == 'DC').sum()),
    Stumpings=('Pick', lambda x: (x.str.strip().str.upper() == 'S').sum()),
    Good_Throws=('Throw', lambda x: (x.str.strip().str.upper() == 'Y').sum()),
    Direct_Hits=('Throw', lambda x: (x.str.strip().str.upper() == 'DH').sum()),
    Run_Outs=('Throw', lambda x: (x.str.strip().str.upper() == 'RO').sum()),
    Missed_Run_Outs=('Throw', lambda x: (x.str.strip().str.upper() == 'MR').sum()),
    Total_Runs_Impact=('Runs', lambda x: pd.to_numeric(x, errors='coerce').sum()),
    Total_PS=('Ball_PS', 'sum')
).reset_index()

print("Fielding Analysis Summary for 3 Players:")
print(summary)

# Save the detailed and summary data
df_filtered.to_csv('fielding_analysis_detailed.csv', index=False)
summary.to_csv('fielding_analysis_summary.csv', index=False)

Fielding Analysis Summary for 3 Players:
     Player Name  Clean_Picks  Catches  Dropped_Catches  Stumpings  \
0      Phil Salt            1        0                0          0   
1  Rilee russouw            0        0                0          0   
2     Yash Dhull            1        0                0          0   

   Good_Throws  Direct_Hits  Run_Outs  Missed_Run_Outs  Total_Runs_Impact  \
0            1            0         0                0                0.0   
1            0            0         0                0                1.0   
2            1            0         0                0                0.0   

   Total_PS  
0       2.0  
1       1.0  
2       2.0  


In [22]:
# Check unique values in Pick and Throw to ensure mapping is correct
print("Unique Pick values:", df['Pick'].dropna().unique())
print("Unique Throw values:", df['Throw'].dropna().unique())

# Recalculate summary using the whole dataset for these 3 players
# Re-checking the weights and mapping based on the provided formula in the prompt
# PS=(CP×WCP)+(GT×WGT)+(C×WC)+(DC×WDC)+(ST×WST)+(RO×WRO )+(MRO×WMRO)+(DH×WDH)+RS

# Let's verify Rilee Russouw's data in the full df
rilee_data = df[df['Player Name'] == 'Rilee russouw']
print("\nRilee Russouw raw data:")
print(rilee_data[['BallCount', 'Pick', 'Throw', 'Runs']])

# Let's count metrics for Rilee specifically based on the prompt's example calculation
# PS=(2*1 CP) + (1*1 GT) + (1*3 C) - (0*3 DC) + (0*3 ST) + (0*3 RO) - (0*2 MRO) + (1*2 DH) + 2 RS = 10
# This implies there are more rows for Rilee than just the one I saw.

# Let's re-run the summary for the whole dataset
summary_full = df[df['Player Name'].isin(players_to_analyze)].groupby('Player Name').apply(lambda x: pd.Series({
    'CP': (x['Pick'].str.strip().str.upper() == 'Y').sum(),
    'C': (x['Pick'].str.strip().str.upper() == 'C').sum(),
    'DC': (x['Pick'].str.strip().str.upper() == 'DC').sum(),
    'ST': (x['Pick'].str.strip().str.upper() == 'S').sum(),
    'GT': (x['Throw'].str.strip().str.upper() == 'Y').sum(),
    'DH': (x['Throw'].str.strip().str.upper() == 'DH').sum(),
    'RO': (x['Throw'].str.strip().str.upper() == 'RO').sum(),
    'MRO': (x['Throw'].str.strip().str.upper() == 'MR').sum(),
    'RS': pd.to_numeric(x['Runs'], errors='coerce').sum()
})).reset_index()

# Calculate PS
summary_full['PS'] = (summary_full['CP'] * 1 + 
                      summary_full['GT'] * 1 + 
                      summary_full['C'] * 3 + 
                      summary_full['DC'] * -3 + 
                      summary_full['ST'] * 3 + 
                      summary_full['RO'] * 3 + 
                      summary_full['MRO'] * -2 + 
                      summary_full['DH'] * 2 + 
                      summary_full['RS'])

print("\nUpdated Summary:")
print(summary_full)

Unique Pick values: ['n' 'Y' 'Run Outs (RO)' '0' '1']
Unique Throw values: ['Y' 'Missed Run Outs (MR)' '0' '1']

Rilee Russouw raw data:
  BallCount Pick Throw Runs
0       0.1    n   NaN    1

Updated Summary:
     Player Name   CP    C   DC   ST   GT   DH   RO  MRO   RS   PS
0      Phil Salt  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0
1  Rilee russouw  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
2     Yash Dhull  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0


C:\Users\DELL\AppData\Local\Temp\ipykernel_11564\4088814058.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_full = df[df['Player Name'].isin(players_to_analyze)].groupby('Player Name').apply(lambda x: pd.Series({


In [23]:
# Check all unique players and their counts to see where the data is
print("Value counts for Player Name:")
print(df['Player Name'].value_counts())

# Let's see all rows for Rilee Russouw
print("\nAll rows for Rilee russouw:")
print(df[df['Player Name'].str.contains('Rilee', case=False, na=False)])

# Let's check the bottom of the file too
print("\nTail of the dataframe:")
print(df.tail(20))

Value counts for Player Name:
Player Name
1                4
Lalit yadav      2
Kuldeep yadav    2
0                2
Rilee russouw    1
Phil Salt        1
Yash Dhull       1
Axer Patel       1
Aman Khan        1
Catches (C)      1
2                1
Name: count, dtype: int64

All rows for Rilee russouw:
  Match No. Innings           Teams    Player Name BallCount  \
0   IPL2367       1  Delhi Capitals  Rilee russouw       0.1   

           Position Pick Throw Runs Overcount  Venue              Stadium  
0  Short mid wicket    n   NaN    1         1  Delhi  Arun Jaitly Stadium  

Tail of the dataframe:
        Match No.           Innings             Teams    Player Name  \
0         IPL2367                 1    Delhi Capitals  Rilee russouw   
1         IPL2367                 1    Delhi Capitals      Phil Salt   
2         IPL2367                 1    Delhi Capitals     Yash Dhull   
3         IPL2367                 1    Delhi Capitals     Axer Patel   
5         IPL2367            

In [25]:
# Read the summary section from the file
# Looking at the tail, it starts at row index 18 (0-indexed) after the header at index 17.
# We'll reload the csv and slice the relevant part.

df_full = pd.read_csv('IPLsampledata.csv', header=None)

# Find where "Player Name" occurs in column 0 of the raw data (it was column index 1 in the cleaned one)
# In df_raw, column 1 had 'Player Name' at index 4 and index 17.
summary_header_idx = 17
summary_data = df_full.iloc[summary_header_idx + 1: summary_header_idx + 8, 1:12]
summary_data.columns = ['Player Name', 'CP', 'GT', 'C', 'DC', 'ST', 'RO', 'MRO', 'DH', 'RS', 'PS']

# Filter for the 3 players
three_players = summary_data[summary_data['Player Name'].isin(['Rilee russouw', 'Phil Salt', 'Yash Dhull'])].copy()

# Ensure all columns are numeric
cols_to_convert = ['CP', 'GT', 'C', 'DC', 'ST', 'RO', 'MRO', 'DH', 'RS', 'PS']
three_players[cols_to_convert] = three_players[cols_to_convert].apply(pd.to_numeric)

print("Selected 3 Players Fielding Performance:")
print(three_players)

# Save the deliverable
three_players.to_csv('cricket_fielding_analysis_summary.csv', index=False)

Selected 3 Players Fielding Performance:
      Player Name  CP  GT  C  DC  ST  RO  MRO  DH  RS  PS
23  Rilee russouw   2   1  1   0   0   0    0   1   2  10
24      Phil Salt   1   2  0   1   0   1    0   0  -1   2


In [26]:
print("All players in summary_data:")
print(summary_data['Player Name'].tolist())

# Re-filter and fix the selection
three_players_list = ['Rilee russouw', 'Phil Salt', 'Yash Dhull']
final_analysis = summary_data[summary_data['Player Name'].isin(three_players_list)].copy()
final_analysis[cols_to_convert] = final_analysis[cols_to_convert].apply(pd.to_numeric)

print("\nFinal Analysis for 3 players:")
print(final_analysis)

# Also create the "complete fielding data" as requested.
# I'll include the ball-by-ball part and the summary.
ball_by_ball = df.head(10) # Just using the available ball data
ball_by_ball.to_csv('fielding_data_balls.csv', index=False)
final_analysis.to_csv('fielding_analysis_3_players.csv', index=False)

All players in summary_data:
[nan, nan, nan, 'Sample Performance Matrix', 'Player Name', 'Rilee russouw', 'Phil Salt']

Final Analysis for 3 players:
      Player Name  CP  GT  C  DC  ST  RO  MRO  DH  RS  PS
23  Rilee russouw   2   1  1   0   0   0    0   1   2  10
24      Phil Salt   1   2  0   1   0   1    0   0  -1   2


In [27]:
# Print the tail end of the raw dataframe starting from row 17
print(df_full.iloc[17:30, :])

     0                          1                 2                 3   \
17  NaN                        NaN               NaN               NaN   
18  NaN                        NaN               NaN               NaN   
19  NaN                        NaN               NaN               NaN   
20  NaN                        NaN               NaN               NaN   
21  NaN  Sample Performance Matrix               NaN               NaN   
22  NaN                Player Name  Clean Picks (CP)  Good Throws (GT)   
23  NaN              Rilee russouw                 2                 1   
24  NaN                  Phil Salt                 1                 2   
25  NaN                 Yash Dhull                 3                 1   
26  NaN                 Axer Patel                 2                 3   
27  NaN                Lalit yadav                 1                 2   
28  NaN                  Aman Khan                 4                 1   
29  NaN              Kuldeep yadav    